In [ ]:
import pandas as pd

df = pd.read_csv('../selected-ann.csv')

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
import json
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = df['content']
Y = df['score']
i = 0
for train_index, test_index in skf.split(X, Y):
    train_dict = []
    val_dict = []
    for index in train_index:
        train_dict.append({'content': "作文：" + df['content'][index], 'summary': "评分：" + str(df['score'][index]) + "分"})
    for index in test_index:
        val_dict.append({'content': "作文：" + df['content'][index], 'summary': "评分：" + str(df['score'][index]) + "分"})
    print(len(train_dict), len(val_dict))
    train_str = ""
    val_str = ""
    for item in train_dict:
        train_str += json.dumps(item, ensure_ascii=False) + '\n'
    for item in val_dict:
        val_str += json.dumps(item, ensure_ascii=False) + '\n'
    with open('data/train' + str(i) + '.json', 'w', encoding='utf-8') as f:
        f.write(train_str)
    with open('data/val' + str(i) + '.json', 'w', encoding='utf-8') as f:
        f.write(val_str)
    i += 1


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("./ChatGLM2-6B/ptuning/output/0-chatglm2-6b-pt-128-2e-2/checkpoint-3000", trust_remote_code=True)
model = AutoModel.from_pretrained("./ChatGLM2-6B/ptuning/output/0-chatglm2-6b-pt-128-2e-2/checkpoint-3000", trust_remote_code=True).half().cuda()
model = model.eval()
import pandas as pd
import json
import os

In [ ]:
# read data/val0.json
import json
file = open('data/val0.json', 'r', encoding='utf-8')
lines = file.readlines()

index = 0
for line in lines:
    obj = json.loads(line.strip())
    content = obj['content']
    summary = obj['summary']
    print(content)
    response, history = model.chat(tokenizer, content, history=[])
    print(response)
    json.dump({'content': content, 'summary': summary, 'response': response, 'history': history}, open('result/0' + str(index) + '.json', 'a', encoding='utf8'), ensure_ascii=False)
    index = index + 1
    print(index)

In [ ]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

In [ ]:

# calculate the QWK
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import json

# draw confusion matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


def qwk(seq, step):
    y_true = []
    y_pred = []
    for i in range(600):
        y_true.append([])
        y_pred.append([])
    for index in [0, 1, 2, 3, 4]:
        for time in [1, 2, 3, 4, 5]:
            filename = "ChatGLM2-6B/ptuning/output/" + str(index) + "-chatglm2-6b-pt-" + str(seq) + "-1e-2/" + str(step) + "/" + str(time) + "/generated_predictions.txt"
            res = open(filename).read()
            idx = 0
            for r in res.split('\n'):
                if r!="":
                    rJSON = json.loads(r)
                    labels = rJSON['labels']
                    predict = rJSON['predict']
                    y_true[120*index + idx].append(int(labels[3]))
                    have = False
                    for score in [0,1,2,3,4,5]:
                        if str(score) in predict:
                            y_pred[120*index + idx].append(score)
                            have = True
                            break
                    if not have:
                        y_pred[120*index + idx].append(0)
                    idx = idx + 1
    def avgWithoutMinMax(arr):
        return round((sum(arr) - max(arr) - min(arr)) / (len(arr) - 2))

    for i in range(len(y_true)):
        y_true[i] = avgWithoutMinMax(y_true[i])
        y_pred[i] = avgWithoutMinMax(y_pred[i])
    # draw confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # print(cm)
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

for seq in [128, 256, 512, 1024]:
    print(seq)
    for step in [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000]:
        print(qwk(seq, step))